In [1]:
from pyspark import SparkContext, SparkConf


from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.feature import HashingTF, IDF, Tokenizer 
from pyspark.ml import Pipeline

import numpy as np

In [2]:
!pip3 install pymongo

    100% |████████████████████████████████| 501kB 854kB/s ta 0:00:01


In [3]:
# create_spark_session
conf = SparkConf()
conf.setAppName('amazon detect fake comment')
conf.setMaster("spark://spark-master:7077")
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)

In [4]:
import os
import pandas as pd
import pymongo

In [104]:
def readData():
    mongo_uri = os.environ.get('MONGO_URI',"mongodb://mongo:27017/")
    myclient = pymongo.MongoClient(mongo_uri)
    mydb = myclient["big-data"]
    mycol = mydb["reviews"]
    data = mycol.find()
    df =  pd.DataFrame(list(data))
    return df

In [105]:
df = readData()

In [106]:
df.head()

,_id,has_image,num_helpful,review_content,review_rating,review_title,reviewer_avg_rating,reviewer_avg_word_count,reviewer_num_review,reviewer_num_verified,reviewer_ranking,reviewer_trust,trust,verified_purchase,review_id,product_id,reviewer_id
0,5fdacc38fd213b96509bf0bb,[False],0,body scrub,5,Great,4.3,8.7,21,21,4119896,71,100,[True],NaN,NaN,NaN
1,5fdacc38fd213b96509bf0bc,[False],0,good product!,5,Five Stars,5,2.3,14,14,22504054,33,0,[True],NaN,NaN,NaN
2,5fdacc38fd213b96509bf0bd,[False],0,"Never write reviews , but this stuff is amazin...",5,Amazing !!!,5,13,2,0,19442408,0,0,[False],NaN,NaN,NaN
3,5fdacc38fd213b96509bf0be,[False],0,"You may see great results for a while, but tru...",1,Witch hazel is awful for your skin no matter what,3,27,2,1,63277988,29,0,[False],NaN,NaN,NaN
4,5fdacc38fd213b96509bf0bf,[True],0,Me fascina como me deja la piel,5,Muy rico olor que queda en la piel,5,0,1,1,63277988,100,100,[True],NaN,NaN,NaN


In [107]:
df = df.drop(columns = ['review_id', 'product_id', 'reviewer_id'])

In [108]:
df.head()

,_id,has_image,num_helpful,review_content,review_rating,review_title,reviewer_avg_rating,reviewer_avg_word_count,reviewer_num_review,reviewer_num_verified,reviewer_ranking,reviewer_trust,trust,verified_purchase
0,5fdacc38fd213b96509bf0bb,[False],0,body scrub,5,Great,4.3,8.7,21,21,4119896,71,100,[True]
1,5fdacc38fd213b96509bf0bc,[False],0,good product!,5,Five Stars,5,2.3,14,14,22504054,33,0,[True]
2,5fdacc38fd213b96509bf0bd,[False],0,"Never write reviews , but this stuff is amazin...",5,Amazing !!!,5,13,2,0,19442408,0,0,[False]
3,5fdacc38fd213b96509bf0be,[False],0,"You may see great results for a while, but tru...",1,Witch hazel is awful for your skin no matter what,3,27,2,1,63277988,29,0,[False]
4,5fdacc38fd213b96509bf0bf,[True],0,Me fascina como me deja la piel,5,Muy rico olor que queda en la piel,5,0,1,1,63277988,100,100,[True]


In [109]:
df = df.dropna()

In [110]:
df.head()

,_id,has_image,num_helpful,review_content,review_rating,review_title,reviewer_avg_rating,reviewer_avg_word_count,reviewer_num_review,reviewer_num_verified,reviewer_ranking,reviewer_trust,trust,verified_purchase
0,5fdacc38fd213b96509bf0bb,[False],0,body scrub,5,Great,4.3,8.7,21,21,4119896,71,100,[True]
1,5fdacc38fd213b96509bf0bc,[False],0,good product!,5,Five Stars,5,2.3,14,14,22504054,33,0,[True]
2,5fdacc38fd213b96509bf0bd,[False],0,"Never write reviews , but this stuff is amazin...",5,Amazing !!!,5,13,2,0,19442408,0,0,[False]
3,5fdacc38fd213b96509bf0be,[False],0,"You may see great results for a while, but tru...",1,Witch hazel is awful for your skin no matter what,3,27,2,1,63277988,29,0,[False]
4,5fdacc38fd213b96509bf0bf,[True],0,Me fascina como me deja la piel,5,Muy rico olor que queda en la piel,5,0,1,1,63277988,100,100,[True]


In [111]:
df = df.reset_index(drop=True)

In [112]:
for i in range(len(df['review_content'])):
  a = df.at[i, 'review_content']
  if type(a) != str:
    df = df.drop(index=i)
    print(i)

9051


In [113]:
df = df.reset_index(drop=True)

In [114]:
df = df.drop(columns = ['_id'])

In [115]:
df.head(1210)

,has_image,num_helpful,review_content,review_rating,review_title,reviewer_avg_rating,reviewer_avg_word_count,reviewer_num_review,reviewer_num_verified,reviewer_ranking,reviewer_trust,trust,verified_purchase
0,[False],0,body scrub,5,Great,4.3,8.7,21,21,4119896,71,100,[True]
1,[False],0,good product!,5,Five Stars,5,2.3,14,14,22504054,33,0,[True]
2,[False],0,"Never write reviews , but this stuff is amazin...",5,Amazing !!!,5,13,2,0,19442408,0,0,[False]
3,[False],0,"You may see great results for a while, but tru...",1,Witch hazel is awful for your skin no matter what,3,27,2,1,63277988,29,0,[False]
4,[True],0,Me fascina como me deja la piel,5,Muy rico olor que queda en la piel,5,0,1,1,63277988,100,100,[True]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,[True],0,Very nice !!!!! So cool! Recommend!,5,Cool,5,5,1,1,63277988,25,25,[True]
1206,[False],0,This product has a lovely scent but I don't re...,2,"Great scent, Not for those who need a strong d...",4.9,20,36,33,4327885,71,100,[True]
1207,[True],1,This deodorant destroyed my underarm skin. You...,1,Terrible for sensitive skin,4.5,46.1,42,41,707986,88,100,[True]
1208,[False],0,I have tried several deodorants and my under ...,5,Great product.,4.3,16.3,23,23,63277988,63,100,[True]


In [116]:

def toFloat(name):
    for i in range(len(df[name])):
      a = df.at[i, name]
      if type(a) == str:
        a = a.replace(",", "")
        if len(a) == 0:
            a = 0.0
      df.at[i, name] = float(a)
    


In [117]:
df = df.reset_index(drop=True)

In [122]:
toFloat('reviewer_avg_rating')
toFloat('reviewer_num_review')
toFloat('reviewer_num_verified')
toFloat('reviewer_avg_word_count')
toFloat('reviewer_ranking')
toFloat('review_rating')

In [126]:
def toStr(name):
    for i in range(12000, len(df)):
        a = df.at[i, name]
        if type(a) == bool:
            if a:
                df.at[i, name] = "[True]"
            else:
                df.at[i, name] = "[False]"
            


In [127]:
toStr('has_image')
toStr('verified_purchase')

In [120]:
def toInt(name):
    for i in range(12000, len(df)):
        a = df.at[i, name]
        if type(a) == str:
            df.at[i, name] = int(a)

In [123]:
toInt('num_helpful')
toInt('trust')

toInt('reviewer_trust')

In [128]:
spark_dataframe = spark.createDataFrame(df)

In [129]:
spark_dataframe.printSchema()

root
 |-- has_image: string (nullable = true)
 |-- num_helpful: long (nullable = true)
 |-- review_content: string (nullable = true)
 |-- review_rating: double (nullable = true)
 |-- review_title: string (nullable = true)
 |-- reviewer_avg_rating: double (nullable = true)
 |-- reviewer_avg_word_count: double (nullable = true)
 |-- reviewer_num_review: double (nullable = true)
 |-- reviewer_num_verified: double (nullable = true)
 |-- reviewer_ranking: double (nullable = true)
 |-- reviewer_trust: long (nullable = true)
 |-- trust: long (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [130]:
spark_dataframe.na.drop(how='any').show()

+---------+-----------+--------------------+-------------+--------------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----+-----------------+
|has_image|num_helpful|      review_content|review_rating|        review_title|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|trust|verified_purchase|
+---------+-----------+--------------------+-------------+--------------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----+-----------------+
|  [False]|          0|          body scrub|          5.0|               Great|                4.3|                    8.7|               21.0|                 21.0|       4119896.0|            71|  100|           [True]|
|  [False]|          0|       good product!|          5.0|          Five Stars|                5.0|             

In [131]:
spark_dataframe.count()

12562

In [132]:
spark_dataframe.describe()

DataFrame[summary: string, has_image: string, num_helpful: string, review_content: string, review_rating: string, review_title: string, reviewer_avg_rating: string, reviewer_avg_word_count: string, reviewer_num_review: string, reviewer_num_verified: string, reviewer_ranking: string, reviewer_trust: string, trust: string, verified_purchase: string]

In [133]:
# return the trust to form 0 or 1
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [134]:
def convert_trust(x):
	return 1 if str(x) > str(50) else 0

In [135]:
trust_udf = udf(convert_trust, IntegerType())
spark_dataframe = spark_dataframe.withColumn('trust', trust_udf('trust'))


In [136]:
def convert_verified_purchase(x):
	return 0 if x == "[False]" else 1

In [137]:
verified_purchase_udf = udf(convert_verified_purchase, IntegerType())
spark_dataframe = spark_dataframe.withColumn('verified_purchase', verified_purchase_udf('verified_purchase'))

#review_df.show(5)

In [138]:
# change False to 0 and True to 1
def convert_has_image(x):
	return 0 if str(x) == "[False]" else 1

In [139]:
has_image_udf = udf(convert_has_image, IntegerType())
spark_dataframe = spark_dataframe.withColumn('has_image', has_image_udf('has_image'))


In [140]:
# text preprocessing
import gensim.parsing.preprocessing as gsp
from pyspark.sql.types import StringType
from gensim import utils

In [141]:
filters = [
    gsp.strip_tags, 
    gsp.strip_punctuation,
    gsp.strip_multiple_whitespaces,
    gsp.strip_numeric,
    gsp.remove_stopwords, 
    gsp.strip_short, 
    gsp.stem_text
]

In [142]:
# merge_content_title(
from pyspark.sql import functions as sf
spark_dataframe = spark_dataframe.withColumn("content_title", sf.concat(sf.col("review_content"), sf.lit(' '), sf.col("review_title")))

In [143]:
# normalize text change from icon to text 
# such as :)) to smile

import re
import pandas as pd

In [144]:
import emoji
emoji.demojize('Python is 👍')

'Python is :thumbs_up:'

In [145]:
spark_dataframe.show(1)

+---------+-----------+--------------+-------------+------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----+-----------------+----------------+
|has_image|num_helpful|review_content|review_rating|review_title|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|trust|verified_purchase|   content_title|
+---------+-----------+--------------+-------------+------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----+-----------------+----------------+
|        0|          0|    body scrub|          5.0|       Great|                4.3|                    8.7|               21.0|                 21.0|       4119896.0|            71|    0|                1|body scrub Great|
+---------+-----------+--------------+-------------+------------+-------------------+---------------

In [146]:

def give_emoji_free_text(x):
  text = x[13]
  if type(text) == str:
    text.replace("\"", "")
    text = emoji.demojize(text)
  return (x[0],x[1], x[3], x[5], x[6], x[7], x[8], x[9], x[10], x[12], text, x[11])
  

In [147]:
#  replace emoji to text
temp_rdd = spark_dataframe.rdd.map(lambda x: give_emoji_free_text(x))
temp_df = temp_rdd.toDF(["has_image","num_helpful",
  "review_rating","reviewer_avg_rating",
  "reviewer_avg_word_count", 
  "reviewer_num_review" ,"reviewer_num_verified",
  "reviewer_ranking","reviewer_trust", 
  "verified_purchase", " content_title","trust"])

In [148]:
temp_df.printSchema()

root
 |-- has_image: long (nullable = true)
 |-- num_helpful: long (nullable = true)
 |-- review_rating: double (nullable = true)
 |-- reviewer_avg_rating: double (nullable = true)
 |-- reviewer_avg_word_count: double (nullable = true)
 |-- reviewer_num_review: double (nullable = true)
 |-- reviewer_num_verified: double (nullable = true)
 |-- reviewer_ranking: double (nullable = true)
 |-- reviewer_trust: long (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |--  content_title: string (nullable = true)
 |-- trust: long (nullable = true)



In [149]:
def clean_text(x):
  review_content_tittle = x[10]
  review_content_tittle  = review_content_tittle.lower()
  review_content_tittle = utils.to_unicode(review_content_tittle)

  for filter in filters:
    review_content_tittle = filter(review_content_tittle)

  return (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], review_content_tittle, x[11])

In [150]:
# cleaned_data_create 

input_rdd = temp_df.rdd.map(lambda x: clean_text(x))
input_df = input_rdd.toDF(["has_image","num_helpful",
  "review_rating","reviewer_avg_rating",
  "reviewer_avg_word_count", 
  "reviewer_num_review" ,"reviewer_num_verified",
  "reviewer_ranking","reviewer_trust", 
  "verified_purchase", "content_title",
  "trust"])

In [151]:
input_df.show(1)

+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+----------------+-----+
|has_image|num_helpful|review_rating|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|verified_purchase|   content_title|trust|
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+----------------+-----+
|        0|          0|          5.0|                4.3|                    8.7|               21.0|                 21.0|       4119896.0|            71|                1|bodi scrub great|    0|
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+----------------+-----+
only showing to

In [152]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.feature import MinMaxScaler

In [153]:
# split train and test 
train_df, test_df = input_df.randomSplit([0.8, 0.2])

In [154]:
train_df.count()

10088

In [155]:
# Word2Vec
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline

In [156]:
tokenizer = Tokenizer(inputCol="content_title",outputCol="words")
w2v =  Word2Vec(vectorSize=5, minCount=0, inputCol="words", outputCol="features_text")

In [157]:
doc2vec_pipeline = Pipeline(stages=[tokenizer,w2v])
doc2vec_model = doc2vec_pipeline.fit(train_df)
train_df = doc2vec_model.transform(train_df)
test_df = doc2vec_model.transform(test_df)

In [158]:
print('train_df after fit and transform')
train_df.show(1)

train_df after fit and transform
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|has_image|num_helpful|review_rating|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|verified_purchase|       content_title|trust|               words|       features_text|
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|        0|          0|          1.0|                1.0|                    0.0|                1.0|                  1.0|     6.3277988E7|             0|                1|fue que pedí pout...|    0|[fue, que, pedí, ...|[-0.0701545627255

In [159]:
train_df.printSchema()

root
 |-- has_image: long (nullable = true)
 |-- num_helpful: long (nullable = true)
 |-- review_rating: double (nullable = true)
 |-- reviewer_avg_rating: double (nullable = true)
 |-- reviewer_avg_word_count: double (nullable = true)
 |-- reviewer_num_review: double (nullable = true)
 |-- reviewer_num_verified: double (nullable = true)
 |-- reviewer_ranking: double (nullable = true)
 |-- reviewer_trust: long (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- content_title: string (nullable = true)
 |-- trust: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features_text: vector (nullable = true)



In [160]:
# normalize other feature to range 0..1
# first transform each cell to a vector
# second run pipeline to transfrom it by MinMax Scaler
columns_to_scale = ["has_image","num_helpful",
  "review_rating","reviewer_avg_rating",
  "reviewer_avg_word_count", 
  "reviewer_num_review" ,"reviewer_num_verified",
  "reviewer_ranking","reviewer_trust", 
  "verified_purchase"]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec", handleInvalid="skip") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)


In [161]:
train_df.show(1)

+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|has_image|num_helpful|review_rating|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|verified_purchase|       content_title|trust|               words|       features_text|
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|        0|          0|          1.0|                1.0|                    0.0|                1.0|                  1.0|     6.3277988E7|             0|                1|fue que pedí pout...|    0|[fue, que, pedí, ...|[-0.0701545627255...|
+---------+-----------+-----

In [162]:
test_df.show(1)

+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|has_image|num_helpful|review_rating|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|verified_purchase|       content_title|trust|               words|       features_text|
+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+
|        0|          0|          1.0|                1.0|                    0.0|                1.0|                  1.0|     2.9399501E7|             0|                1|resibi est produc...|    0|[resibi, est, pro...|[-0.0029271449893...|
+---------+-----------+-----

In [163]:
from pyspark.sql.types import FloatType, DoubleType
def convert_string(x):
	return float(x)

In [164]:
def convert_string_to_int(x):
	return int(x)

In [165]:
review_rating_df = udf(convert_string, FloatType())
train_df = train_df.withColumn('review_rating', review_rating_df('review_rating'))

reviewer_avg_rating_df = udf(convert_string, FloatType())
reviewer_avg_word_count_df = udf(convert_string, FloatType())
reviewer_num_review_df = udf(convert_string_to_int, IntegerType())
reviewer_num_verified_df = udf(convert_string_to_int, IntegerType())
reviewer_ranking_df = udf(convert_string_to_int, IntegerType())
reviewer_trust_df = udf(convert_string, FloatType())


train_df = train_df.withColumn('reviewer_avg_rating', review_rating_df('reviewer_avg_rating') )
train_df = train_df.withColumn('reviewer_avg_word_count', review_rating_df('reviewer_avg_word_count') )
train_df = train_df.withColumn('reviewer_num_review', review_rating_df('reviewer_num_review') )
train_df = train_df.withColumn('reviewer_num_verified', review_rating_df('reviewer_num_verified') )
train_df = train_df.withColumn('reviewer_ranking', review_rating_df('reviewer_ranking') )
train_df = train_df.withColumn('reviewer_avg_rating', review_rating_df('reviewer_avg_rating') )
train_df = train_df.withColumn('reviewer_trust', review_rating_df('reviewer_trust') )



In [166]:
review_rating_df = udf(convert_string, FloatType())
test_df = test_df.withColumn('review_rating', review_rating_df('review_rating'))

reviewer_avg_rating_df = udf(convert_string, FloatType())
reviewer_avg_word_count_df = udf(convert_string, FloatType())
reviewer_num_review_df = udf(convert_string_to_int, IntegerType())
reviewer_num_verified_df = udf(convert_string_to_int, IntegerType())
reviewer_ranking_df = udf(convert_string_to_int, IntegerType())
reviewer_trust_df = udf(convert_string, FloatType())


test_df = test_df.withColumn('reviewer_avg_rating', review_rating_df('reviewer_avg_rating') )
test_df = test_df.withColumn('reviewer_avg_word_count', review_rating_df('reviewer_avg_word_count') )
test_df = test_df.withColumn('reviewer_num_review', review_rating_df('reviewer_num_review') )
test_df = test_df.withColumn('reviewer_num_verified', review_rating_df('reviewer_num_verified') )
test_df = test_df.withColumn('reviewer_ranking', review_rating_df('reviewer_ranking') )
test_df = test_df.withColumn('reviewer_avg_rating', review_rating_df('reviewer_avg_rating') )
test_df = test_df.withColumn('reviewer_trust', review_rating_df('reviewer_trust') )



In [167]:
train_df.printSchema()

root
 |-- has_image: long (nullable = true)
 |-- num_helpful: long (nullable = true)
 |-- review_rating: float (nullable = true)
 |-- reviewer_avg_rating: float (nullable = true)
 |-- reviewer_avg_word_count: float (nullable = true)
 |-- reviewer_num_review: float (nullable = true)
 |-- reviewer_num_verified: float (nullable = true)
 |-- reviewer_ranking: float (nullable = true)
 |-- reviewer_trust: float (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- content_title: string (nullable = true)
 |-- trust: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features_text: vector (nullable = true)



In [168]:
test_df.printSchema()

root
 |-- has_image: long (nullable = true)
 |-- num_helpful: long (nullable = true)
 |-- review_rating: float (nullable = true)
 |-- reviewer_avg_rating: float (nullable = true)
 |-- reviewer_avg_word_count: float (nullable = true)
 |-- reviewer_num_review: float (nullable = true)
 |-- reviewer_num_verified: float (nullable = true)
 |-- reviewer_ranking: float (nullable = true)
 |-- reviewer_trust: float (nullable = true)
 |-- verified_purchase: long (nullable = true)
 |-- content_title: string (nullable = true)
 |-- trust: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features_text: vector (nullable = true)



In [169]:
scalerModel = pipeline.fit(train_df)
train_df = scalerModel.transform(train_df)
test_df = scalerModel.transform(test_df)

In [170]:

# print('train_df after fit and transform')

# test_df.select('reviewer_trust_scaled').show(1)

# here we have bug
# vectorassembler use when concat many column into a vector
# 1 2 3 --> [1,2,3]
# but we want 
# [1], [2], [3] ---> [1,2,3]


#third concat it to a column named featured_nun
vector_assembler_num = VectorAssembler(\
  inputCols=[
  "has_image_scaled","num_helpful_scaled",
  "review_rating_scaled","reviewer_avg_rating_scaled",
  "reviewer_avg_word_count_scaled", 
  "reviewer_num_review_scaled" ,"reviewer_num_verified_scaled",
  "reviewer_ranking_scaled","reviewer_trust_scaled", 
  "verified_purchase_scaled"
  ],\
  outputCol="features_num")

In [171]:
train_df = vector_assembler_num.transform(train_df)

In [172]:
train_df.show(1)

+---------+-----------+-------------+-------------------+-----------------------+-------------------+---------------------+----------------+--------------+-----------------+--------------------+-----+--------------------+--------------------+-------------+---------------+-----------------+-----------------------+---------------------------+-----------------------+-------------------------+--------------------+------------------+---------------------+----------------+------------------+--------------------+--------------------------+------------------------------+--------------------------+----------------------------+-----------------------+---------------------+------------------------+--------------------+
|has_image|num_helpful|review_rating|reviewer_avg_rating|reviewer_avg_word_count|reviewer_num_review|reviewer_num_verified|reviewer_ranking|reviewer_trust|verified_purchase|       content_title|trust|               words|       features_text|has_image_vec|num_helpful_vec|review_rat

In [173]:
train_df.select("features_text").show()

+--------------------+
|       features_text|
+--------------------+
|[-0.0701545627255...|
|[-0.0597543617089...|
|[-0.1862397529184...|
|[-0.0489047524053...|
|[-0.0089826285839...|
|[-0.1509825497865...|
|[0.01430998295545...|
|[0.01389714516699...|
|[-0.0065002058981...|
|[-0.4396339226514...|
|[-0.1012028406063...|
|[-0.2147730216383...|
|[0.11754906177520...|
|[0.03154478718837...|
|[-0.3695423478881...|
|[-0.0694851167500...|
|[0.12660136965236...|
|[-0.0613030387709...|
|[-0.1972733300179...|
|[-0.0693318744500...|
+--------------------+
only showing top 20 rows



In [174]:

test_df = vector_assembler_num.transform(test_df)


In [175]:
test_df.select("features_num").show()
test_df.select("features_text").show()

+--------------------+
|        features_num|
+--------------------+
|(10,[6,7,9],[1.99...|
|(10,[4,6,7,9],[0....|
|(10,[4,6,7,9],[0....|
|(10,[4,7],[0.0050...|
|(10,[4,6,7,9],[0....|
|(10,[4,6,7,8,9],[...|
|(10,[4,6,7,9],[0....|
|(10,[4,7],[0.0122...|
|(10,[4,5,6,7,8],[...|
|(10,[4,7],[0.0231...|
|(10,[4,7],[0.0354...|
|(10,[4,7],[0.0412...|
|(10,[4,7],[0.0426...|
|[0.0,0.0,0.0,9.48...|
|[0.0,0.0,0.0,1.70...|
|[0.0,0.0,0.0,2.46...|
|[0.0,0.0,0.0,2.46...|
|[0.0,0.0,0.0,3.22...|
|[0.0,0.0,0.0,3.41...|
|[0.0,0.0,0.0,3.79...|
+--------------------+
only showing top 20 rows

+--------------------+
|       features_text|
+--------------------+
|[-0.0029271449893...|
|[-0.0152293462306...|
|[0.08918498549610...|
|[-0.0633669737726...|
|[-0.0930243059992...|
|[-0.3843827247619...|
|[-0.3130229388674...|
|[-0.0106201805174...|
|[-0.2630627408404...|
|[-0.2797066381822...|
|[-0.3047843351960...|
|[-0.2385845463033...|
|[-0.3934323363006...|
|[-0.0029919128865...|
|[-0.1623531427573...|
|[-0.127

In [176]:

data_array_train_num =  np.array(train_df.select("features_num").collect())
data_array_test_num =  np.array(test_df.select("features_num").collect())
data_array_train_text = np.array(train_df.select("features_text").collect())
data_array_test_text = np.array(test_df.select("features_text").collect())


In [177]:
print(data_array_train_num.shape)
# (a, 1, 10)
print(data_array_test_num.shape)
# (b,1, 10)


(10088, 1, 10)
(2474, 1, 10)


In [178]:
print(data_array_train_num)

[[[0.         0.         0.         ... 0.81467874 0.         1.        ]]

 [[0.         0.         0.         ... 0.30712995 0.         1.        ]]

 [[0.         0.         0.         ... 0.29355389 0.         0.        ]]

 ...

 [[1.         0.1025641  0.         ... 0.0044743  0.95       1.        ]]

 [[1.         0.19711538 1.         ... 0.00309355 0.71       1.        ]]

 [[1.         0.3482906  0.         ... 0.03463149 0.48       1.        ]]]


In [179]:
x = data_array_train_num.shape
y = data_array_test_num.shape
x_text = data_array_train_text.shape
y_text = data_array_test_text.shape

In [180]:
data_array_train_num.resize(x[0], x[2])
data_array_test_num.resize(y[0], y[2])

In [181]:
print(data_array_train_num.shape)
print(data_array_test_num.shape)

print(data_array_train_text.shape)
print(data_array_test_text.shape)


(10088, 10)
(2474, 10)
(10088, 1, 5)
(2474, 1, 5)


In [182]:
print(data_array_train_num)

[[0.         0.         0.         ... 0.81467874 0.         1.        ]
 [0.         0.         0.         ... 0.30712995 0.         1.        ]
 [0.         0.         0.         ... 0.29355389 0.         0.        ]
 ...
 [1.         0.1025641  0.         ... 0.0044743  0.95       1.        ]
 [1.         0.19711538 1.         ... 0.00309355 0.71       1.        ]
 [1.         0.3482906  0.         ... 0.03463149 0.48       1.        ]]


In [183]:

data_array_train_text.resize(x_text[0], x_text[2])
data_array_test_text.resize(y_text[0], y_text[2])


print(data_array_train_text.shape)
print(data_array_test_text.shape)


(10088, 5)
(2474, 5)


In [184]:
features_train = np.concatenate((data_array_train_text, data_array_train_num), axis=1)
features_test = np.concatenate((data_array_test_text, data_array_test_num), axis=1)

print(features_train.shape)
print(features_test.shape)

label_train = np.array(train_df.select("trust").collect())
label_test = np.array(test_df.select("trust").collect())


print(label_train.shape)
print(label_test.shape)


train_np = np.concatenate((label_train, features_train), axis=1)
test_np = np.concatenate((label_test, features_test), axis=1)


(10088, 15)
(2474, 15)
(10088, 1)
(2474, 1)


In [185]:
print(train_np.shape)
print(test_np.shape)
# convert numpy array to data frame 
# and add to train_df and test_df
# import pandas as pd 
# print('------------------------------------------')
from pyspark.ml.linalg import Vectors
train_df_pre = map(lambda x: (int(x[0]), Vectors.dense(x[1:])), train_np)
train_df_final = spark.createDataFrame(train_df_pre, schema=["label", "features"])

test_df_pre = map(lambda x: (int(x[0]), Vectors.dense(x[1:])), test_np)
test_df_final = spark.createDataFrame(test_df_pre, schema=["label", "features"])


(10088, 16)
(2474, 16)


In [186]:
train_df_final

DataFrame[label: bigint, features: vector]

In [187]:
test_df_final.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[-0.0029271449893...|
|    0|[-0.0152293462306...|
|    0|[0.08918498549610...|
|    0|[-0.0633669737726...|
|    0|[-0.0930243059992...|
+-----+--------------------+
only showing top 5 rows



In [188]:
train_df_final.show(20)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[-0.0701545627255...|
|    0|[-0.0597543617089...|
|    0|[-0.1862397529184...|
|    0|[-0.0489047524053...|
|    0|[-0.0089826285839...|
|    0|[-0.1509825497865...|
|    0|[0.01430998295545...|
|    0|[0.01389714516699...|
|    0|[-0.0065002058981...|
|    0|[-0.4396339226514...|
|    0|[-0.1012028406063...|
|    0|[-0.2147730216383...|
|    0|[0.11754906177520...|
|    0|[0.03154478718837...|
|    0|[-0.3695423478881...|
|    0|[-0.0694851167500...|
|    0|[0.12660136965236...|
|    0|[-0.0613030387709...|
|    0|[-0.1972733300179...|
|    0|[-0.0693318744500...|
+-----+--------------------+
only showing top 20 rows



In [189]:
np_array_train_ft =  np.array(train_df_final.select("features").collect())
np_array_test_ft =  np.array(test_df_final.select("features").collect())

In [190]:
np_array_train_label =  np.array(train_df_final.select("label").collect())
np_array_test_label =  np.array(test_df_final.select("label").collect())

In [191]:
np_shape_test = np_array_test_ft.shape

In [192]:
np_shape_train = np_array_train_ft.shape

In [193]:
np_array_train_ft.resize(np_shape_train[0], np_shape_train[2])
np_array_test_ft.resize(np_shape_test[0], np_shape_train[2])


In [194]:
np_array_train_ft.shape

(10088, 15)

In [195]:
np_array_train_label.shape

(10088, 1)

In [196]:
!pip3 install sklearn

    100% |████████████████████████████████| 6.8MB 159kB/s ta 0:00:01
    100% |████████████████████████████████| 307kB 852kB/s ta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [197]:
# import important packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# import pandas_profiling

In [198]:
classifier = RandomForestClassifier()

# Train the model using the training sets
classifier.fit(np_array_train_ft, np_array_train_label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier()

In [199]:
y_pred = classifier.predict(np_array_test_ft)

In [200]:
print("Accuracy:", accuracy_score(np_array_test_label, y_pred))

Accuracy: 0.9919159256265158


In [201]:
train_df_final.show(20)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[-0.0701545627255...|
|    0|[-0.0597543617089...|
|    0|[-0.1862397529184...|
|    0|[-0.0489047524053...|
|    0|[-0.0089826285839...|
|    0|[-0.1509825497865...|
|    0|[0.01430998295545...|
|    0|[0.01389714516699...|
|    0|[-0.0065002058981...|
|    0|[-0.4396339226514...|
|    0|[-0.1012028406063...|
|    0|[-0.2147730216383...|
|    0|[0.11754906177520...|
|    0|[0.03154478718837...|
|    0|[-0.3695423478881...|
|    0|[-0.0694851167500...|
|    0|[0.12660136965236...|
|    0|[-0.0613030387709...|
|    0|[-0.1972733300179...|
|    0|[-0.0693318744500...|
+-----+--------------------+
only showing top 20 rows



In [202]:
spark.stop()
sc.stop()